In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!wget -q https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget -q https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget -q https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget -q https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

In [3]:
# 필요한 패키지 임포트
import os
import numpy as np
import pandas as pd
import datetime
import sys
import zipfile
import modeling
import optimization
import run_classifier
import tokenization
 
from tokenization import FullTokenizer
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
 
import tensorflow_hub as hub
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

In [4]:
# BERT모델과 tokenization의 파라미터 지정
sess = tf.Session()
 
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
max_seq_length = 128 #최장 토큰 길이

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# 파일 로드
train_df = pd.read_csv('/content/drive/My Drive/VAIV/BERT/train/train.csv', index_col='id')
val_df = pd.read_csv('/content/drive/My Drive/VAIV/BERT/valid/valid.csv', index_col='id')
test_df = pd.read_csv('/content/drive/My Drive/VAIV/BERT/test/test.csv', index_col='id')

In [7]:
label_encoder = LabelEncoder().fit(pd.concat([train_df['label'], val_df['label']]))
#문자 label을 숫자로 encoding 시킨다.

In [8]:
X_train_val, X_test = pd.concat([train_df['text'], val_df['text']]).values, test_df['text'].values

In [9]:
y_train_val = label_encoder.fit_transform(pd.concat([train_df['label'], val_df['label']]))
#train data set에 대한 scaling

In [10]:
X_train, X_val, y_train, y_val = train_test_split(
        X_train_val,y_train_val, test_size=0.1, random_state=0, stratify = y_train_val
        )
#random state : data 분할 시 shuffle random seed
#stratify : 지정한 data 분할 유지, 나누어진 데이터셋도 기존 클래스 비율과 같아야 함.

In [11]:
train_text = X_train
train_text = [' '.join(t.split()[0:max_seq_length]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = y_train
 
val_text = X_val
val_text = [' '.join(t.split()[0:max_seq_length]) for t in val_text]
val_text = np.array(val_text, dtype=object)[:, np.newaxis]
val_label = y_val
 
test_text = X_test
test_text = [' '.join(t.split()[0:max_seq_length]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

#join 하는 부분은 max_seq_length를 넘는 token값을 제거하기 위한 수단
#np.newaxis는 차원을 하나 추가하는 방법인데 train_text[0]으로 출력하면 dtype이, [0][0]으로 출력하면 string만 나오게끔 설정됨 (1차 => 2차원)

#아무튼 위의 과정들은 다 전처리 관련한 것임 (scaling, max_token 넘는거 빼기, 파일 형식 바꿔주기 등등)

In [12]:
X_train[0]

"I got this as a supplement for skin and coat for my dogs. I couldn't tell a differenceat all in their coat or shedding.My mom used the remainder for her dogs and said it did seem to keep fleas off."

In [13]:
import tensorflow as tf
import tensorflow_hub as hub
import os
import re
import numpy as np
from tqdm import tqdm_notebook #진행률 모듈인 듯??
from keras import backend as K
from keras.layers import Layer

Using TensorFlow backend.


In [14]:
class BertLayer(Layer):
    
    def __init__(self, n_fine_tune_layers=10, tf_hub = None, output_representation = 'pooled_output', trainable = False, **kwargs):
        
        self.n_fine_tune_layers = n_fine_tune_layers
        self.is_trainble = trainable
        self.output_size = 768
        self.tf_hub = tf_hub
        self.output_representation = output_representation
        self.supports_masking = True
        
        super(BertLayer, self).__init__(**kwargs)
 
    def build(self, input_shape):
 
        self.bert = hub.Module(
            self.tf_hub,
            trainable=self.is_trainble,
            name="{}_module".format(self.name)
        )
        
        
        variables = list(self.bert.variable_map.values())
        if self.is_trainble:
            # 1 first remove unused layers
            trainable_vars = [var for var in variables if not "/cls/" in var.name]
            
            
            if self.output_representation == "sequence_output" or self.output_representation == "mean_pooling":
                # 1 first remove unused pooled layers
                trainable_vars = [var for var in trainable_vars if not "/pooler/" in var.name]
                
            # Select how many layers to fine tune
            trainable_vars = trainable_vars[-self.n_fine_tune_layers :]
            
            # Add to trainable weights
            for var in trainable_vars:
                self._trainable_weights.append(var)
 
            # Add non-trainable weights
            for var in self.bert.variables:
                if var not in self._trainable_weights:
                    self._non_trainable_weights.append(var)
                
        else:
             for var in variables:
                self._non_trainable_weights.append(var)
                
 
        super(BertLayer, self).build(input_shape)
 
    def compute_mask(self, inputs, mask=None):
        
        if self.output_representation == 'sequence_output':
            inputs = [K.cast(x, dtype="bool") for x in inputs]
            mask = inputs[1]
            
            return mask
        else:
            return None
        
        
    def compute_output_shape(self, input_shape):
        if self.output_representation == "sequence_output":
            return (input_shape[0][0], input_shape[0][1], self.output_size)
        else:
            return (input_shape[0][0], self.output_size)

In [15]:
import keras
 
def build_model(max_seq_length, tf_hub, n_classes, n_fine_tune): 
    in_id = keras.layers.Input(shape=(max_seq_length,), name="input_ids") #token단위
    in_mask = keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = BertLayer(n_fine_tune_layers=n_fine_tune, tf_hub = tf_hub, output_representation = 'mean_pooling', trainable = True)(bert_inputs)
    drop = keras.layers.Dropout(0.3)(bert_output)
    dense = keras.layers.Dense(256, activation='sigmoid')(drop)
    drop = keras.layers.Dropout(0.3)(dense)
    dense = keras.layers.Dense(64, activation='sigmoid')(drop)
    pred = keras.layers.Dense(n_classes, activation='softmax')(dense)
    
    model = keras.models.Model(inputs=bert_inputs, outputs=pred)
    Adam = keras.optimizers.Adam(lr = 0.0005)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam, metrics=['sparse_categorical_accuracy'])
    model.summary()
 
    return model

In [20]:
def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)
n_classes = len(label_encoder.classes_)
n_fine_tune_layers = 48
n_classes

6

In [21]:
model = build_model(max_seq_length, bert_path, n_classes, n_fine_tune_layers)
 
# Instantiate variables
initialize_vars(sess)

IndexError: ignored